In [39]:
from pathlib import Path
import configparser
from psycopg2 import connect
import pandas.io.sql as pandasql
from IPython.display import HTML
from psycopg2.extras import execute_values
import requests
from psycopg2.extras import RealDictCursor
import json
import click 
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)


In [3]:
# function for creating query_info_table
def create_query_info_table(layer_name):
    sql = 'CREATE TABLE gis_shared_streets.'+layer_name+'_query_info_table (like gis_shared_streets.query_info_structure)'
    with con:
            with con.cursor() as cur:
                cur.execute(sql)

In [668]:
# Prepare input table 
# All input table should only include streets, if a street is bi-directional then there should be one geometry for each direction. 

sql= '''create view gis_shared_streets.centreline_streets_only as (select geo_id, direction, dir_geom AS geom from gis_shared_streets.centreline_both_dir where FCODE_DESC IN ('Collector','Collector Ramp',
                'Local','Major Arterial','Major Arterial Ramp','Minor Arterial',
                'Minor Arterial Ramp','Pending') ORDER BY geo_id)'''
with con:
    with con.cursor(cursor_factory=RealDictCursor) as cur:
                    cur.execute(sql)


In [44]:
# Use matching function
unmatched_here_snap = matching(10, 0.6, 25, None, 'natalie.routing_street_18_3', 'gis_shared_streets.here_matched_snap','gis_shared_streets.here_query_info_table_snap', 'link_dir',None, con)


matched_rows length:199
0
unmatched_rows length:15
[]


34524

In [43]:
# Matching function 
def matching(search_radius, length_tolerance, bearing_tolerance, previously_unmatched_id, input_table, output_table, query_info_table, primary_key1, primary_key2, con):
    
    """
    This function processes the input table and inserts the sharedstreets API outputs to the output table. 
    It returns a list of the unmatched unique identifers from the input table.
    
    Parameters 
    ----------
    search_radius : int
        Search radius in meters for snapping streets to SharedStreets references (1-100 meters)
        
    length_tolerance : float
        Line length for potential matches specified as a percent of total line length (0-1)
        
    bearing_tolerance : int
        Degrees tolerance to allow for directional street queries (0-360 degrees)
        
    previously_unmatched_id : list
        List of IDs that were unmatched with previous calls to this function. 
        If this is the first time calling the function on a dataset, then the value should be None
        
    input_table : string
        Name of table with geometry that you would like to match to shared streets 
        Things to note:
            1) All input table should only include streets, if a street is bi-directional then there should be one geometry for each direction.
            2) There must be a primary key of one or two columns
            
    output_table : string
        Name of table that the matched rows will be inserted into
        
    query_info_table :string
        Name of the table that records the general information each time we run the function. 
        This includes parameters used, query number, number of matched rows, as well as the total number of rows that we attempted to match.
        This table should be created with the same column as gis_shared_streets.query_info_structure. 
        
    primary_key1 : string
        The primary key of the input table
        
    primary_key2 : string
        The second primary key of the input table (e.g. if primary_key1 is not the unique identifier for the table) 
    
    con : string
        Database connection credentials 
    
    Returns
    -------
    
    unmatched_rows : list
        List of the unmatched unique identifers (primary_key1 or primary_key1 + primary_key2) from the input table
    
    """
    # get query_num from query info table (i.e. the serial value that was just inputted)
    sql_get_query_num = "SELECT GREATEST(query_num) AS query_num FROM {}".format(query_info_table)
    
    with con:
        with con.cursor() as cur:
            cur.execute(sql_get_query_num) 
            query_results = cur.fetchall()
            if query_results == []:
                query_num = 1
            else:
                query_num = query_results[0]['query_num']
            
            
    # If everything got matched
    if previously_unmatched_id == []:
        return "Everything is matched"
    
    
    # If this is the first time we run the function, then count will be the total number of row in the input table
    if previously_unmatched_id is None:    
        count_sql = "SELECT COUNT(*) FROM {}".format(input_table)
            
        with con:
            with con.cursor(cursor_factory=RealDictCursor) as cur:
                    cur.execute(count_sql)
                    count_results = cur.fetchall()
                    count = count_results[0]['count']
                    
                    
    # If this is not the first time we run the function, then count will be the total number of row in the returned unmatched list of unique identifiers                              
    else: 
        count = len(previously_unmatched_id)
            
    
          
    # list for all the ids that do not get matched 
    unmatched_rows = []
    null_matched = []
    
    # If there is more than one primary key
    if primary_key2 is not None:
        for i in range(0, count, 300):
                  
            
            # first time calling function on input table         
            if previously_unmatched_id is None: 

                # two rows make up the primary key of the table
                get_centreline_sql =  '''SELECT 'Feature' as type,  json_build_object('{}', {}, '{}', {}) as properties, ST_AsGeoJson(geom)::json as geometry 
                FROM {}
                ORDER BY {}, {}
                LIMIT 300 OFFSET {}'''.format(primary_key1, str(primary_key1), primary_key2, str(primary_key2), input_table, primary_key1, primary_key2, i)


            # calling function again
            else:

                get_centreline_sql = '''SELECT 'Feature' as type,  json_build_object('{}', {}, '{}', {}) as properties, ST_AsGeoJson(geom)::json as geometry 
                FROM {} WHERE {}||{} IN {} 
                LIMIT 300 OFFSET {}'''.format(primary_key1, primary_key1, primary_key2, primary_key2, input_table, str(primary_key1), str(primary_key2),tuple(previously_unmatched_id), i)

            with con:
                with con.cursor(cursor_factory=RealDictCursor) as cur:
                    cur.execute(get_centreline_sql)
                    results = cur.fetchall()
                    final_json = json.dumps({"type": "FeatureCollection",
                                            "features": results})
            # Send data to SharedStreets API    
            base_url = "https://api.sharedstreets.io/v0.1.0/match/geoms"
            apiKey = "bdd23fa1-7ac5-4158-b354-22ec946bb575"
            params = {'ignoreDirection': 'false',
                      'bearingTolerance' : bearing_tolerance,
                      'searchRadius' : search_radius,
                      'auth' : apiKey,
                      'tileHierarchy': 6,
                      'dataSource': 'osm/planet-181029',
                      'lengthTolerance' : length_tolerance,
                      'snapTopology': 'true', 
                      'snapToIntersections': 'true'
                     }
            headers= {"Content-type": "application/json; charset=UTF-8"}

            r = requests.post(base_url, params = params, headers = headers, json = {"type": "FeatureCollection", "features": results})
            returned_json = r.json()
            if returned_json.get('unmatched') is not None:
                unmatched = returned_json.get('unmatched').get('features')
                for feature in unmatched:

                        unmatched_id1 = feature['properties']['{}'.format(primary_key1)]
                        unmatched_id2 = feature['properties']['{}'.format(primary_key2)]
                        unmatched_rows.append(str(unmatched_id1) + unmatched_id2)
                
             # Send row to database
            if returned_json.get('matched') is not None:
                features = returned_json.get('matched').get('features')
                rows = []
                for feature in features:
                        if  feature['properties']['section'][0] > feature['properties']['section'][1]:
                            feature['properties']['section'][0] = 0
                            feature['properties']['section'][1] = 0

                        if  feature['properties']['referenceId'] is None:
                            null_matched.append((str(feature['properties']['{}'.format(primary_key1)]))+feature['properties']['{}'.format(primary_key2)])
                            continue

                        row = (feature['properties']['referenceId'], 
                                   feature['properties']['fromIntersectionId'],
                                   feature['properties']['toIntersectionId'],
                                   feature['properties']['roadClass'], 
                                   feature['properties']['direction'],
                                   feature['properties']['referenceLength'],
                                   feature['properties']['side'], 
                                   feature['properties'].get('score', None),
                                   '[' + str(feature['properties']['section'][0]) +', '+ str(feature['properties']['section'][1]) +')', 
                                   'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['coordinates'])+')', 
                                   feature['properties']['originalFeature']['properties']['{}'.format(primary_key1)],
                                   feature['properties']['originalFeature']['properties']['{}'.format(primary_key2)], 
                                   query_num)


                        rows.append(row)


                sql='INSERT INTO {} (reference_id, from_intersection, to_intersection, road_class, ss_direction, reference_length, side,score, section, geometry, {}, {}, query_num) VALUES %s'.format(output_table, primary_key1, primary_key2)


                with con:
                    with con.cursor() as cur:
                        execute_values(cur, sql, rows) 

                print(i)
                print("unmatched_rows length:" + str(len(unmatched_rows)))
                print("matched_rows length:" +str(len(rows)))
                
    else:
        for i in range(0, count, 300):
        
            # First time calling function on input table         
            if previously_unmatched_id is None: 

                # two rows make up the primary key of the table
                get_centreline_sql =  '''SELECT 'Feature' as type,  json_build_object('{}', {}) as properties, ST_AsGeoJson(geom)::json as geometry 
                FROM {}
                LIMIT 300 OFFSET {}'''.format(primary_key1, str(primary_key1), input_table, i)


            # calling function again
            else:

                get_centreline_sql = '''SELECT 'Feature' as type,  json_build_object('{}', {}) as properties, ST_AsGeoJson(geom)::json as geometry 
                FROM {} WHERE {} IN {} 
                LIMIT 300 OFFSET {}'''.format(primary_key1, primary_key1, input_table, str(primary_key1), tuple(previously_unmatched_id), i)

            with con:
                with con.cursor(cursor_factory=RealDictCursor) as cur:
                    cur.execute(get_centreline_sql)
                    results = cur.fetchall()
                    final_json = json.dumps({"type": "FeatureCollection",
                                            "features": results})
                    
            # Send data to SharedStreets API    
            base_url = "https://api.sharedstreets.io/v0.1.0/match/geoms"
            apiKey = "bdd23fa1-7ac5-4158-b354-22ec946bb575"
            params = {'ignoreDirection': 'false',
                      'bearingTolerance' : bearing_tolerance,
                      'searchRadius' : search_radius,
                      'auth' : apiKey,
                      'tileHierarchy': 6,
                      'dataSource': 'osm/planet-181029',
                      'lengthTolerance' : length_tolerance,
                      'snapTopology': 'true', 
                      'snapToIntersections': 'true'
                     }
            headers= {"Content-type": "application/json; charset=UTF-8"}

            r = requests.post(base_url, params = params, headers = headers, json = {"type": "FeatureCollection", "features": results})
            returned_json = r.json()
            if returned_json.get('unmatched') is not None:
                unmatched = returned_json.get('unmatched').get('features')
                for feature in unmatched:
                        unmatched_id1 = feature['properties']['{}'.format(primary_key1)]
                        unmatched_rows.append(str(unmatched_id1))
            
             # Send row to database
            if returned_json.get('matched') is None:
                test_json = json.loads(final_json)
                
                for feature in test_json['features']:
                        unmatched_1 = feature['properties']['{}'.format(primary_key1)]
                        unmatched_rows.append(str(unmatched_1))
                    
            if returned_json.get('matched') is not None:
                features = returned_json.get('matched').get('features')
                rows = []
                for feature in features:
                        if  feature['properties']['section'][0] > feature['properties']['section'][1]:
                            feature['properties']['section'][0] = 0
                            feature['properties']['section'][1] = 0

                        if  feature['properties']['referenceId'] is None:
                            null_matched.append(str(feature['properties']['{}'.format(primary_key1)]))
                            continue

                        row = (feature['properties']['referenceId'], 
                                   feature['properties']['fromIntersectionId'],
                                   feature['properties']['toIntersectionId'],
                                   feature['properties']['roadClass'], 
                                   feature['properties']['direction'],
                                   feature['properties']['referenceLength'],
                                   feature['properties']['side'], 
                                   feature['properties'].get('score', None),
                                   '[' + str(feature['properties']['section'][0]) +', '+ str(feature['properties']['section'][1]) +')', 
                                   'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['coordinates'])+')', 
                                   feature['properties']['originalFeature']['properties']['{}'.format(primary_key1)],
                                   None)


                        rows.append(row)



                sql='INSERT INTO {} (reference_id, from_intersection, to_intersection, road_class, ss_direction, reference_length, side,score, section, geometry, {},  query_num) VALUES %s'.format(output_table,primary_key1)


                with con:
                    with con.cursor() as cur:
                        execute_values(cur, sql, rows) 
                print("matched_rows length:" +str(len(rows)))
            
            print(i)
            print("unmatched_rows length:" + str(len(unmatched_rows)))
                
    
    # put info on query into appropriate table
    sql_query_info = 'INSERT INTO {} (query_num, bearing_tolerance, search_radius, length_tolerance, snap_to_intersection, tilehierarchy, total_rows, matched_rows)  VALUES %s'.format(query_info_table)
    query_list = [(query_num, bearing_tolerance, search_radius, length_tolerance, params['snapToIntersections'], params['tileHierarchy'], count,count-len(unmatched_rows))]
   
    with con:
        with con.cursor() as cur:
            execute_values(cur, sql_query_info, query_list) 
    
    
    # update the output table with the correct query number so we can link the output table to the query_num table 
    sql_update_with_query_num = 'UPDATE {} set query_num = {} WHERE query_num IS NULL'.format(output_table, query_num)
    
    with con:
        with con.cursor() as cur:
            cur.execute(sql_update_with_query_num) 

    
    print(null_matched)
    return unmatched_rows